<h2><center> Code de Hamming et stéganographie </center></h2>

**But**

Obtenir pour un mot d'information de taille k, un mot de code de taille n>k qui comporte une part de redondance

**Code correcteur linéaire défini par [n,k,$\delta$] :**

-sous espace vectoriel d'un espace vectoriel de dimension finie sur un corps fini (ici $F_2 = \{0,1\}$) <br/>
-mots de code de taille n<br/>
-mots d'information (donc message) de taille k <br/>
-n>k

**Codage :**

Matrice génératrice G (de taille n lignes k colonnes)<br/>
Mot d'information m.<br/>
Mot de code c.<br/>
$c = Gm$<br/>

Matrice génératrice systématique $G = \begin{pmatrix}I\\P\end{pmatrix}$ où $P$ est la matrice de parité du code (dépend du code).

**Erreur :**

Le mot de code c subit une erreur e (taille n) dans le canal de communication : $y = c+e$


**Décodage :**

Matrice de contrôle H (de taille n-k lignes n colonnes) telle que HxG = 0<br/>
Mot de code avec erreur y.<br/>
Syndrome s.<br/>
$s = Hy$<br/>

Matrice de contrôle systématique $H = \left(P||I\right)$ où $P$ est la matrice de parité du code précédente.

Si $s=0$, alors $y$ est sans erreur, et donc on peut retrouver le mot d'information en prenant les k premiers éléments de $y$.<br/>
Sinon $s$ contient les bits correspondant aux erreurs.

Exemple:


In [13]:
import numpy as np

def log2(n):
    c = 0
    while n>1:
        n //= 2
        c += 1
    return c

def binarize_array(a):
    for i in range(len(a)):
        a[i] = a[i]%2
    return a

def binarize_matrix(m):
    for i in range(len(m)):
        for j in range(len(m[i])):
            m[(i,j)] = m[(i,j)]%2
    return m

def create_hamming(k):
    n = 2**k-1
    m = n-k

    G = np.diag(np.ones(m, dtype=np.int))
    H = np.diag(np.ones(n-m, dtype=np.int))
    P = np.zeros([m,n-m], dtype=np.int)
    o = 0
    for i in range(1,n+1):
        if 2**(log2(i)) != i:
            for j in range(k):
                P[(o,j)] = ((i&(1<<j))>>j)
            o += 1
    return np.concatenate((G,P.T),axis=0),np.concatenate((P.T,H),axis=1)

G,H = create_hamming(3)
print(u"Matrice génératrice :")
print(G)
print("")
print(u"Matrice de contrôle :")
print(H)
print("")

message = [1,0,0,1]
print("Message :")
print(message)
print("Mot de code correspondant :")
c = binarize_array(G.dot(message))
print(c)
print("")
print("Sans erreur, le syndrome est nul :")
y = c
s = binarize_array(H.dot(y))
print(s)
y[2] = 1-y[2]
print("Mot de code avec une erreur :")
print(y)
s = binarize_array(H.dot(y))
print("Syndrome obtenu :")
print(s)
print(u"Le syndrome correspond à la 3ème colonne de H donc l'erreur est en position 3, c'est bien le cas")
print(u"On retrouve notre message en prenant les 4 premiers bits du y corrigé")

Matrice génératrice :
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 1 0 1]
 [1 0 1 1]
 [0 1 1 1]]

Matrice de contrôle :
[[1 1 0 1 1 0 0]
 [1 0 1 1 0 1 0]
 [0 1 1 1 0 0 1]]

Message :
[1, 0, 0, 1]
Mot de code correspondant :
[1 0 0 1 0 0 1]

Sans erreur, le syndrome est nul :
[0 0 0]
Mot de code avec une erreur :
[1 0 1 1 0 0 1]
Syndrome obtenu :
[0 1 1]
Le syndrome correspond à la 3ème colonne de H donc l'erreur est en position 3, c'est bien le cas
On retrouve notre message en prenant les 4 premiers bits du y corrigé


Notre but est de trouver le vecteur d'erreur $e$ qui vérifie :

- $c = x+e$
- $s = m = Hc$

où $x$ est le médium (bloc de taille n), et $m$ est le message à cacher.<br/>
C'est à dire en combinant les expressions $e$ tel que $He = m-Hx$ ce qui se fait facilement.

Exemple :

In [23]:
m = [1,0,1]
print("Message à cacher :")
print(m)
print("")

x = [1,0,1,1,0,0,1]
print("Bloc de couverture :")
print(x)
print("")

print("m-Hx = ")
print(binarize_array(m-H.dot(x)))
print("")

print(u"Cela correspond à la 1ère colonne de H :")
print(H)
print("")

print(u"Donc en inversant le premier bit du support on obtient quelque chose qui va se déchiffrer en notre message secret :")
y = x
y[0] = 1-y[0]
print(u"Support modifié y : ")
print(y)
print("Hy : ")
print()

Message à cacher :
[1, 0, 1]

Bloc de couverture :
[1, 0, 1, 1, 0, 0, 1]

m-Hx = 
[1 1 0]

Cela correspond à la ème colonne de H :
[[1 1 0 1 1 0 0]
 [1 0 1 1 0 1 0]
 [0 1 1 1 0 0 1]]
